In [6]:
# Import and data read

import numpy as np
import tensorflow as tf
import nltk
from collections import Counter
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import tensorflow_text as text
import tensorflow_hub as hub


with open('reviews.txt', 'r') as f:
  reviews = f.read()
with open('labels.txt', 'r') as f:
  labels = f.read()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Remove punctuation

reviews = "".join([char for char in reviews if char not in string.punctuation])

reviews = reviews.split('\n')
labels = labels.split('\n')

In [3]:
# Tokenization, Lemmatization, Stemming. Label numerical encoding

import itertools

reviews_tokenized = []
for review in reviews:
  splitted_review = nltk.word_tokenize(review)
  splitted_review = [WordNetLemmatizer().lemmatize(w) for w in splitted_review]
  splitted_review = [PorterStemmer().stem(w).strip() for w in splitted_review]
  reviews_tokenized.append(splitted_review)
  
reviews_unrolled = list(itertools.chain(*reviews_tokenized))
  
labels = [1 if label == "positive" else 0 for label in labels]

In [4]:
# Remove empty reviews and the corresponding labels

empty_idx = []
for i, review in enumerate(reviews_tokenized):
  if len(review) == 0:
    empty_idx.append(i)
    
for i in empty_idx:
  reviews_tokenized.pop(i)
  reviews.pop(i)
  labels.pop(i)

In [5]:
# Create vocabulary, word2index reference and convert the reviews into numerical form

vocab_size = 10000

word_counter = Counter(reviews_unrolled)
# word_counter = dict(word_counter.most_common(vocab_size))
word2index = {k:i for i,k in enumerate(word_counter.keys(), start = 3)}

reviews_int = []
for review in reviews_tokenized:
  cur_review = [1]
  for word in review:
    if word in word2index.keys():
      cur_review.append(word2index[word])
    else:
      cur_review.append(2)
  reviews_int.append(cur_review)

In [6]:
# Pad sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_reviews = pad_sequences(reviews_int, maxlen = 500, padding = 'pre', truncating = 'pre')

In [ ]:
# Train test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reviews, labels, test_size = 0.2, random_state = 1)

X_train = np.array(X_train)
X_test = np.array(X_test)

y_train = np.array(y_train).reshape(20000, 1)
y_test = np.array(y_test).reshape(5000, 1)

preprocessor = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")

X_train = preprocessor(X_train)['input_word_ids']
X_test = preprocessor(X_test)['input_word_ids']

In [36]:
# Define the model

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Lambda, GlobalMaxPool1D, BatchNormalization, Dense, RNN, GRU, LSTM, TimeDistributed, Bidirectional, Activation, Embedding, Input, Conv1D, Dropout
import tensorflow as tf
import keras.backend as K
import tensorflow_hub as hub

sample_sentence = 'This is a bad movie'

# embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# embed_samples = embed([sample_sentence, 'This is my second sentence'])


# sentence_encoder_layer = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',
#                                          input_shape = [],
#                                          dtype=tf.string,
#                                          trainable = False)

# sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2", 
#                                         output_shape=[256],
#                                         input_shape=[], 
#                                         dtype=tf.string, trainable = False)


# inputs = Input(shape = ())
# x = preprocessor(inputs)
# x = Dense(128, activation = 'relu')(x['['input_word_ids']'])
# outputs = Dense(1, activation = 'sigmoid')(x)

# model = Model(inputs = inputs, outputs = outputs)


model = Sequential([
  Dense(128, activation = 'relu'),
  Dense(128, activation = 'relu'),
  Dense(1, activation = 'sigmoid')
])


# inputs = Input(shape = (X_train.shape[1:]))
# mask = tf.keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
# x = Embedding(input_dim = vocab_size, output_dim = 128, input_length = 200)(inputs)
# x = Conv1D(filters = 200, kernel_size = 13, strides = 1, padding = 'same', activation = 'relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(dropout_rate)(x)
# x = GRU(128, return_sequences = True)(x)
# x = BatchNormalization()(x)
# x = Dropout(dropout_rate)(x)
# x = GRU(128, return_sequences = False)(x)
# x = BatchNormalization()(x)
# x = Dropout(dropout_rate)(x)
# x = Dense(512, activation = 'relu')(x)
# x = Dropout(dropout_rate)(x)
# outputs = Dense(1, activation = 'sigmoid')(x)



# model = Model(inputs = inputs, outputs = outputs)

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])

# model.summary()

In [37]:
# # Train the model

history = model.fit(X_train, y_train, epochs = 100, batch_size = 128, validation_data = (X_test, y_test))

Epoch 1/100


ValueError: in user code:

    File "C:\Users\User\anaconda3\envs\newtensorflow\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\User\anaconda3\envs\newtensorflow\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\User\anaconda3\envs\newtensorflow\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\User\anaconda3\envs\newtensorflow\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\User\anaconda3\envs\newtensorflow\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\User\AppData\Local\Temp\__autograph_generated_filetpbww9j2.py", l

In [20]:
# import tensorflow_hub as hub

# embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
# embeddings = embed(["cat is on the mat", "dog is in the fog"])

In [13]:


# preprocessor = hub.KerasLayer(
#     "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

In [14]:
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")


In [15]:
encodings = preprocessor(X_train[0:3])

In [16]:
encodings['input_word_ids']

<tf.Tensor: shape=(3, 128), dtype=int32, numpy=
array([[   101,  14985,  16460,  24990,  32291,  14997,  14985,  19750,
         16090,    170, 175173,  46983,  14999,  75298,  14985,  40790,
         31782,  15308,  14997,    170, 191046,  24674,  15206,  73584,
         18688,  35545,    170,  74357,  29161,  15014,  78152, 211814,
        310398,  14986,  18566,  38764,  16895, 140901,  15444,  86831,
         15127,  19155,  78152, 211814,  14999, 124087, 282517,  22897,
        217557,  14983,  15036,  34264,  15206,  14985,  79526,  14997,
         14985,  19750,  57247,  15141,  30510,  15053,  19559,  16596,
         15636,  15384,  15061,  17072,  15061,  15932,  22285,  14981,
        332259, 360845,  15277,  46186,  15272,  19750,  15206,  14985,
         50742, 394578,  14985,  16191,  19132,  15021,  44163,  14986,
         16512,  33865,  23099,  15205,  21117,  14999,  15808,    189,
         21466,  15272,  19750,  21466, 332259, 360845,  15277,  14999,
         37735, 